In [1]:
import pandas as pd
import numpy as np
import re
import word2number

In [3]:
df = pd.read_csv("~/Downloads/Messy Animal Data - Animal Dataset (1).csv")

In [7]:
df.head(10)

,ID,Animal,Height (cm),Weight (kg),Color,Lifespan (years),Diet,Habitat,Predators,Average Speed (km/h),Countries Found,Conservation Status,Family,Gestation Period (days),Top Speed (km/h),Social Structure,Offspring per Birth
0,1,Aardvark,105-130,40-65,Grey,20-30,NaN,"Savannas, Grasslands","Lions, Hyenas",40,Africa,Least Concern,Orycteropodidae,210-240,40,Solitary,1
1,2,Aardwolf,40-50,8-14,Yellow-brown,10-12,NaN,"Grasslands, Savannas","Lions, Leopards",24-30,Eastern and Southern Africa,Least Concern,Hyaenidae,90,40,Solitary,2-5
2,3,African Elephant,270-310,2700-6000,Grey,60-70,Herbivore,"Savannah, Forest","Lions, Hyenas",25,Africa,Vulnerable,Elephantidae,640-660,40,Herd-based,1
3,4,African Lion,80-110,120-250,Tan,10-14,Carnivore,"Grasslands, Savannas","Hyenas, Crocodiles",58,Africa,Vulnerable,Felidae,98-105,80,Group-based,2-4 (usually)
4,5,African Wild Dog,75-80,18-36,Multicolored,10-12,Carnivore,Savannahs,"Lions, Hyenas",fifty-six,Sub-Saharan Africa,Endangered,Canidae,70,56,Group-based,10-12
5,6,Alpine Ibex,67-101,19-120,Brown,15-20,Herbivore,"Mountains, Alpine","Wolves, Golden Eagles",56-64,European Alps,Least Concern,Bovidae,150,56-64,Group-based,1
6,7,Amazon Rainforest Frog,2-13,Up to 0.5,Various,4-17,Insectivore,Amazon Rainforest,"Birds, Snakes",0.1-1,Amazon Rainforest,Not Evaluated,Dendrobatidae,10-15,0.8,Solitary,Up to 40
7,8,American Bison,152-186,"318-1,000",Brown,15-20,Herbivore,"Grasslands, Plains","Wolves, Grizzly Bears",40-56,North America,Near Threatened,Bovidae,270-290,56,Group-based,1
8,9,NaN,52-91,22-41,"Brown, White",15-20,Insectivore,"Grasslands, Forests","Jaguars, Pumas",24,Central and South America,Least Concern,Myrmecophagidae,190-210,24,Solitary,1
9,10,Arabian Horse,140-160,380-1000,Various,25-30,Herbivore,"Middle East, North Africa","Humans, Predators",65,"Middle East, North Africa",Not Applicable,Equidae,330-342,65,Herd-based,1


In [9]:
from word2number import w2n
import pandas as pd
import re

cols = ['Animal', 'Average Speed (km/h)', 'Diet']

for col in cols:
    df[col] = df[col].astype(str).str.strip()

for col in cols:
    df[col] = df[col].replace(['nan', 'NaN', ''], pd.NA)
df = df.dropna(subset=cols)

df['Diet'] = df['Diet'].str.lower()
df = df[df['Diet'].isin(['carnivore', 'herbivore', 'omnivore'])]

def words_to_number(text):
    text = str(text).lower().strip()
    if re.match(r'^[\d.\-]+$', text):
        return text
    try:
        return str(w2n.word_to_num(text))
    except ValueError:
        return pd.NA

df['Average Speed (km/h)'] = df['Average Speed (km/h)'].apply(words_to_number)

def speed_to_avg(val):
    val = str(val)
    if '-' in val:
        parts = val.split('-')
        try:
            return sum(float(p) for p in parts) / len(parts)
        except ValueError:
            return pd.NA
    try:
        return float(val)
    except ValueError:
        return pd.NA

df['Average Speed (km/h)'] = df['Average Speed (km/h)'].apply(speed_to_avg)

df = df.dropna(subset=cols)

df_clean = df[cols].rename(columns={
    'Animal': 'name',
    'Average Speed (km/h)': 'speed',
    'Diet': 'diet'
}).reset_index(drop=True)

C:\Users\bowen\AppData\Local\Temp\ipykernel_43076\2067944253.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Diet'] = df['Diet'].str.lower()


In [13]:
df_clean.head(10)

,name,speed,diet
0,African Elephant,25.0,herbivore
1,African Lion,58.0,carnivore
2,African Wild Dog,56.0,carnivore
3,Alpine Ibex,60.0,herbivore
4,American Bison,48.0,herbivore
5,Arabian Horse,65.0,herbivore
6,Arabian Oryx,55.0,herbivore
7,Arctic Fox,60.0,omnivore
8,Asian Elephant,40.0,herbivore
9,Atlantic Spotted Dolphin,52.0,carnivore


In [15]:
df_clean.to_csv('sample_animals.csv', index=False)